In [2]:
from ollama import chat

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [2]:
skills = [
    "stack(blue_block, green_block)",
    "stack(blue_block, red_block)",
    "stack(green_block, red_block)",
    "stack(green_block, blue_block)",
    "stack(red_block, blue_block)",
    "stack(red_block, green_block)",
    "pick_first(blue_block)",
    "pick_first(green_block)",
    "pick_first(red_block)",
    "done()"
]

In [ ]:
def skill_list(instruction, skills, model):
    S = []  # Skill list
    n = 0   # Step counter
    selected_skill=""
    while selected_skill!="done()":
        C = {}  # Likelihood scores for each skill
        for skill_name in skills:
            prompt = f"""
You are a robotic arm responsible for stacking colored blocks in order.  
You can select from these skills to complete the instruction:
1. pick_first(block): Must be the first command. The argument specifies the block at the bottom of the stack.  
2. stack(bottom_block, top_block): Places 'top_block' on top of 'bottom_block'.  
3. done(): Must be the last command after the stack is completed.

Given the instruction: {instruction}, and the previously executed skills: {S}, determine the likelihood of selecting the skill {skill_name} as the next step.

Rules:  
- Skills must be in the correct order of execution.
- Minimize the number of steps.  
- If no skills have been selected (empty array), you must choose the first skill.  
- Return a decimal number between 0 and 1, rounded to two decimal places.  
- Respond only with number, no text.""" 
            
            response = chat(model=model, messages=[{"role": "user", "content": prompt}])
            print(response["message"]["content"])
            likelihood = float(response["message"]["content"].strip())
            C[skill_name] = likelihood
        # Select the skill with the highest likelihood
        selected_skill = max(C, key=C.get)
        print(f"Selected skill: {selected_skill}")
        S.append(selected_skill)
        n += 1

    return S

instruction = "Stack the blocks in the order of: green, blue, red."
skill_list = skill_list(instruction, skills, model="llama3.1")
print("Generated Skill List with Parameters:", skill_list)

# ChatGPT

In [3]:
def skill_list_gpt(instruction, skills):
    S = []  # Skill list
    n = 0   # Step counter
    selected_skill=""
    while selected_skill!="done()":
        C = {}  # Likelihood scores for each skill
        for skill_name in skills:
            prompt = f"""
You are a robotic arm responsible for stacking colored blocks in order.  
You can select from these skills to complete the instruction:

1. pick_first(block): Must be the first command. The argument specifies the block at the bottom of the stack.  
2. stack(bottom_block, top_block): Places 'top_block' on top of 'bottom_block'.  
3. done(): Must be the last command after the stack is completed.

Given the instruction: {instruction}, and the previously executed skills: {S}, determine the likelihood of selecting the skill {skill_name} as the next step.

Rules:  
- Skills must be in the correct order of execution.
- Minimize the number of steps.  
- If no skills have been selected (empty array), you must choose the first skill.  
- Return a decimal number between 0 and 1, rounded to two decimal places.  
- Respond only with number, no text."""
            
            completion = client.chat.completions.create(model="gpt-4o", messages=[{"role": "user", "content": prompt}])
            print(completion.choices[0].message.content)
            likelihood = float(completion.choices[0].message.content.strip())
            C[skill_name] = likelihood
        # Select the skill with the highest likelihood
        selected_skill = max(C, key=C.get)
        print(f"Selected skill: {selected_skill}")
        S.append(selected_skill)
        n += 1

    return S

In [5]:
# Generate the skill list
instruction = "Place the red block on the green block."
skill_list = skill_list_gpt(instruction, skills)

0.00
0.0
0.00
0.00
0.00
0.00
0.00
1.0
1.00
0.00
Selected skill: pick_first(green_block)
0.0
0.00
1.00
0.0
0.0
0.0
0.0
0.00
0.00
0.00
Selected skill: stack(green_block, red_block)
0.00
0.00
1.00
0.0
0.00
0.0
0.00
0.0
0.00
1.00
Selected skill: stack(green_block, red_block)
0.00
0.00
0.00
0.0
0.00
1.0
0.0
0.0
0.00
0.00
Selected skill: stack(red_block, green_block)
0.00
0.00
0.0
0.00
0.0
0.0
0.00
0.00
0.00
0.5
Selected skill: done()
